# Toronto Neighbourhood

### Question 1

Importing required libraries

In [43]:
!pip install bs4
!pip install lxml

In [44]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import lxml

Scrape data from the Wikipedia page and store it in a Pandas dataframe

In [45]:
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

wiki = BeautifulSoup(url,'html')
table = wiki.find('table',{'class':'wikitable'})

Postcode = []
Borough = []
Neighbourhood = []

for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        Postcode.append(cells[0].find(text=True))
        Borough.append(cells[1].find(text=True))
        Neighbourhood.append(cells[2].find(text=True))

df = pd.DataFrame()
df['Postcode'] = Postcode
df['Borough'] = Borough
df['Neighbourhood'] = Neighbourhood

df = df.replace('\n','', regex=True)
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


Process the dataframe according to rules given:

* Remove rows with Boroughs undefined

In [46]:
df.drop(df[df['Borough']=='Not assigned'].index, inplace=True)
df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
160,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


In [47]:
df.loc[df[df['Neighbourhood']=='Not assigned\n'].index,'Neighbourhood']=df.loc[df[df['Neighbourhood']=='Not assigned\n'].index,'Borough']
df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
160,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


* Combine areas with multiple Neighbourhoods into one row

In [48]:
df_new = pd.DataFrame({'Neighbourhood_count' : df.groupby( ['Postcode','Borough'] ).size()}).reset_index()
df.reset_index()

Neighbourhood_full = []

for i in range(len(df_new)):
    string = ''
    for j in range(len(df)):
        if df_new.iloc[i,0]==df.iloc[j,0]:
            if string=='':
                string = df.iloc[j,2].rstrip()
            else:
                string = string + ', ' + df.iloc[j,2].rstrip()
    Neighbourhood_full.append(string)
    
df_new.drop(columns='Neighbourhood_count',inplace=True)
df_new['Neighbourhood'] = Neighbourhood_full

df_new

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...


In [49]:
df_new.shape

(103, 3)

### Question 2

Install Geocoder

In [50]:
!pip install geocoder

Import Geocoder to obtain the geographical coordinates of the postal codes

In [51]:
import geocoder

Latitude = []
Longitude = []

for i in range(len(df_new)):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        postal_code = df_new.iloc[i,0]
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    Latitude.append(lat_lng_coords[0])
    Longitude.append(lat_lng_coords[1])

KeyboardInterrupt: 

Since the Geocoder is not returning the coordinates correctly, the information is obtained from the csv file provided

In [52]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


Read the data into a Pandas dataframe and view the data

In [53]:
df_coordinates = pd.read_csv('Geospatial_Coordinates.csv')
df_coordinates

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Include the Latitude and Longitude data into the Toronto neighbourhoods dataframe

In [54]:
df_new['Latitude']=df_coordinates['Latitude']
df_new['Longitude']=df_coordinates['Longitude']
df_new

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...,43.688905,-79.554724
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.739416,-79.588437


### Question 3

Import packages to visualize locations on maps

In [55]:
import folium

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

# All requested packages already installed.



Fetch address of Toronto from geolocator

In [56]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="cn_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Visualization of the neighbourhoods of Toronto on map

In [57]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Setup Foursquare API credentials

In [58]:
CLIENT_ID = 'ZX3Q0H2O1WBZJFKQ4VMLSFLNIIYDRP2J2AHC5SI5W1FO0NA0' # your Foursquare ID
CLIENT_SECRET = 'HCDOA5E4SX22345S51I4UFBSYJCRDHNUSWNE34P4402MMMJZ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZX3Q0H2O1WBZJFKQ4VMLSFLNIIYDRP2J2AHC5SI5W1FO0NA0
CLIENT_SECRET:HCDOA5E4SX22345S51I4UFBSYJCRDHNUSWNE34P4402MMMJZ


Fetch nightlife spots in the neighbourhoods

In [121]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            'Nightlife Spot',
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [122]:
toronto_venues = getNearbyVenues(names=df_new['Neighbourhood'],latitudes=df_new['Latitude'],longitudes=df_new['Longitude'])

Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview / Henry Farm / Oriole
Bayview Village
York Mills / Silver Hills
Willowdale / Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor / Wilson Heights / Downsview North
Northwood Park / York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill / Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West / Riverdale
India Bazaar / The Beaches 

In [123]:
print(toronto_venues.shape)
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
toronto_venues.head()

(568, 7)
There are 20 uniques categories.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
1,Cedarbrae,43.773136,-79.239476,The Avenue,43.772437,-79.241962,Lounge
2,Birch Cliff / Cliffside West,43.692657,-79.264848,Andrew's Place,43.692953,-79.262791,Pub
3,Birch Cliff / Cliffside West,43.692657,-79.264848,Dirt Floor Brewing,43.693725,-79.267130,Brewery
4,Birch Cliff / Cliffside West,43.692657,-79.264848,Royal Canadian Legion #73,43.689980,-79.266920,Bar


Identify 10 most popular types of nightlife spots in the neighbourhoods

In [124]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

num_top_venues = 10

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Wine Bar,Hookah Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
1,Alderwood / Long Branch,Pub,Wine Bar,Hookah Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
2,Bathurst Manor / Wilson Heights / Downsview North,Bar,Whisky Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar
3,Bedford Park / Lawrence Manor East,Pub,Wine Bar,Hookah Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
4,Berczy Park,Pub,Bar,Cocktail Bar,Beer Bar,Nightlife Spot,Nightclub,Lounge,Gay Bar,Beer Garden,Brewery
...,...,...,...,...,...,...,...,...,...,...,...
56,Westmount,Bar,Whisky Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar
57,Wexford / Maryvale,Hookah Bar,Wine Bar,Whisky Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
58,Willowdale,Wine Bar,Gastropub,Lounge,Hookah Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gay Bar
59,Woodbine Heights,Bar,Brewery,Whisky Bar,Beer Bar,Beer Garden,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar


Run k-means clutering to divide neighbourhoods into clusters for analysis

In [125]:
from sklearn.cluster import KMeans
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_new

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.dropna(subset=['Cluster Labels'],inplace=True)

toronto_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,1.0,Bar,Whisky Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4.0,Lounge,Wine Bar,Hookah Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
9,M1N,Scarborough,Birch Cliff / Cliffside West,43.692657,-79.264848,4.0,Pub,Bar,Brewery,Hookah Bar,Beer Bar,Beer Garden,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
10,M1P,Scarborough,Dorset Park / Wexford Heights / Scarborough To...,43.757410,-79.273304,0.0,Brewery,Wine Bar,Whisky Bar,Beer Bar,Beer Garden,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar
11,M1R,Scarborough,Wexford / Maryvale,43.750072,-79.295849,3.0,Hookah Bar,Wine Bar,Whisky Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,M8W,Etobicoke,Alderwood / Long Branch,43.602414,-79.543484,2.0,Pub,Wine Bar,Hookah Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
94,M9B,Etobicoke,West Deane Park / Princess Gardens / Martin Gr...,43.650943,-79.554724,0.0,Brewery,Wine Bar,Whisky Bar,Beer Bar,Beer Garden,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar
95,M9C,Etobicoke,Eringate / Bloordale Gardens / Old Burnhamthor...,43.643515,-79.577201,4.0,Bar,Lounge,Hookah Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
99,M9P,Etobicoke,Westmount,43.696319,-79.532242,1.0,Bar,Whisky Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar


Visualization of clusters on map

In [126]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    colour = int(cluster-1)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[colour],
        fill=True,
        fill_color=rainbow[colour],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Brewery

In [127]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,0.0,Brewery,Wine Bar,Whisky Bar,Beer Bar,Beer Garden,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar
36,East York,0.0,Bar,Brewery,Whisky Bar,Beer Bar,Beer Garden,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar
38,East York,0.0,Sports Bar,Brewery,Wine Bar,Hookah Bar,Beer Bar,Beer Garden,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
81,York,0.0,Brewery,Wine Bar,Whisky Bar,Beer Bar,Beer Garden,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar
87,East Toronto,0.0,Brewery,Wine Bar,Whisky Bar,Beer Bar,Beer Garden,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar
94,Etobicoke,0.0,Brewery,Wine Bar,Whisky Bar,Beer Bar,Beer Garden,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar


#### Whiskey bar

In [128]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1.0,Bar,Whisky Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar
18,North York,1.0,Bar,Brewery,Whisky Bar,Beer Bar,Beer Garden,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar
23,North York,1.0,Bar,Whisky Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar
28,North York,1.0,Bar,Whisky Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar
64,Central Toronto,1.0,Bar,Whisky Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar
68,Downtown Toronto,1.0,Bar,Whisky Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar
74,York,1.0,Bar,Whisky Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar
77,West Toronto,1.0,Bar,Cocktail Bar,Wine Bar,Sports Bar,Gastropub,Pub,Lounge,Brewery,Speakeasy,Beer Bar
80,York,1.0,Bar,Whisky Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar
83,West Toronto,1.0,Bar,Whisky Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar,Hookah Bar


#### Wine bar

In [129]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,2.0,Pub,Wine Bar,Hookah Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
49,Central Toronto,2.0,Pub,Sports Bar,Wine Bar,Hookah Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub
53,Downtown Toronto,2.0,Pub,Brewery,Lounge,Gastropub,Wine Bar,Hookah Bar,Beer Bar,Beer Garden,Cocktail Bar,Dive Bar
62,North York,2.0,Pub,Wine Bar,Hookah Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
65,Central Toronto,2.0,Pub,Wine Bar,Hookah Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
89,Etobicoke,2.0,Pub,Wine Bar,Hookah Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar


#### Hookah bar

In [130]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,3.0,Hookah Bar,Wine Bar,Whisky Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
26,North York,3.0,Hookah Bar,Wine Bar,Whisky Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
27,North York,3.0,Hookah Bar,Wine Bar,Whisky Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
71,North York,3.0,Sports Bar,Wine Bar,Hookah Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
72,North York,3.0,Hookah Bar,Sports Bar,Pub,Wine Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub
88,Etobicoke,3.0,Sports Bar,Wine Bar,Hookah Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar


#### Pub

In [131]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,4.0,Lounge,Wine Bar,Hookah Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
9,Scarborough,4.0,Pub,Bar,Brewery,Hookah Bar,Beer Bar,Beer Garden,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
12,Scarborough,4.0,Lounge,Wine Bar,Hookah Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
22,North York,4.0,Wine Bar,Gastropub,Lounge,Hookah Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gay Bar
24,North York,4.0,Wine Bar,Gastropub,Lounge,Hookah Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gay Bar
29,North York,4.0,Bar,Lounge,Hookah Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gastropub,Gay Bar
35,East York,4.0,Bar,Gastropub,Whisky Bar,Beer Bar,Beer Garden,Brewery,Cocktail Bar,Dive Bar,Gay Bar,Hookah Bar
41,East Toronto,4.0,Bar,Pub,Nightlife Spot,Brewery,Lounge,Wine Bar,Sports Bar,Sake Bar,Nightclub,Speakeasy
42,East Toronto,4.0,Brewery,Pub,Nightlife Spot,Wine Bar,Hookah Bar,Beer Bar,Beer Garden,Cocktail Bar,Dive Bar,Gastropub
43,East Toronto,4.0,Brewery,Gastropub,Wine Bar,Gay Bar,Dive Bar,Bar,Nightlife Spot,Nightclub,Lounge,Karaoke Bar
